In [129]:
import utils.constants as const
import utils.pathbuilder as upb
import os
import keras
import random
import csv
from PIL import Image
import numpy as np
import skimage.io
from skimage.color import rgba2rgb, rgb2gray, gray2rgb
keras.__version__

'2.1.3'

In [16]:
### Get samples
images_id = [x for x in os.listdir(const.STAGE1_TRAIN_PATH)]
image_id = images_id[5]

image_path = os.path.join(const.STAGE1_TRAIN_PATH, image_id, 'images', '{}.png'.format(image_id))
mask_paths = os.path.join(const.STAGE1_TRAIN_PATH, image_id, 'masks', '*.png')

image = gray2rgb(rgb2gray(skimage.io.imread(image_path)))
masks = skimage.io.imread_collection(mask_paths).concatenate()
mask = np.zeros(image.shape[:2], np.uint16)

for mask_idx in range(masks.shape[0]):
    mask[masks[mask_idx] > 0] = 1

image = np.uint8(image * 255)
if image.mean() > 255 / 2:
    image = 255 - image
    
print(image.shape)
print(mask.shape)

(360, 360, 3)
(360, 360)


In [17]:
# Adding padding to image and mask
padding = 47
padd_image = apply_padding(image)
padd_mask = apply_padding(mask)
print(padd_image.shape)
print(padd_mask.shape)

(460, 460, 3)
(460, 460)


In [130]:
# Counting the number of training examples in one image
w, h, _ = image.shape
cont_positive_samples = 0
cont_negative_samples = 0

for x in range(w):
    for y in range(h):
        sample = padd_mask[x - padding: x + padding + 1,y - padding : y + padding + 1]
        isValidForTraining = np.sum(sample) > 0
        if isValidForTraining:
            cont_positive_samples += (padd_mask[x][y] / 255. == 1)
            cont_negative_samples += (padd_mask[x][y] / 255. == 0)
    
rate_positive_negative = cont_positive_samples * 100. / cont_negative_samples

all_zeros_added = False
positive_taken = 0
negative_taken = 0
training_examples = []
label_examples = []
for x in range(w):
    for y in range(h):
        sample = padd_mask[x - padding: x + padding + 1,y - padding : y + padding + 1]
        isValidForTraining = np.sum(sample) > 0
        if isValidForTraining:
            if (padd_mask[x][y] / 255. == 0):
                factor = random.randint(0, 1000)
                if factor < rate_positive_negative * 10:
                    negative_taken += 1
                    #img_sample = padd_image[x - padding: x + padding + 1,y - padding : y + padding + 1, :]
                    img_sample = padd_image[x: x + 2 * padding + 1, y : y + 2 * padding + 1, :] / 255.
                    training_examples.append(img_sample)
                    label_examples.append(np.array([0,1]))
            else:
                positive_taken += 1
                #img_sample = padd_image[x - padding: x + padding + 1, y - padding : y + padding + 1, :]
                img_sample = padd_image[x: x + 2 * padding + 1, y : y + 2 * padding + 1, :] / 255.
                training_examples.append(img_sample)
                label_examples.append(np.array([1,0]))
        else:
            if all_zeros_added == False:
                all_zeros_added = True
                img_sample = padd_image[x: x + 2 * padding + 1, y : y + 2 * padding + 1, :] / 255.
                training_examples.append(img_sample)
                label_examples.append(np.array([0,1]))

new_rate = positive_taken * 100. / negative_taken
train = np.array(training_examples)
labels = np.array(label_examples)
#train = train.reshape(train.shape[0], train.shape[1], train.shape[2], 1)
print("------------------------------------------------")
print("Positive taken : ", positive_taken)
print("Negative taken : ", negative_taken)
print("Total taken    : ", positive_taken + negative_taken)
print("Train len      : ", len(training_examples))
print("Train shape    : ", train.shape)
print("Label len      : ", len(label_examples))
print("Label shape    : ", labels.shape)
print("New rate       : %.2f%%"%new_rate)

------------------------------------------------
Positive taken :  28344
Negative taken :  28382
Total taken    :  56726
Train len      :  56727
Train shape    :  (56727, 95, 95, 3)
Label len      :  56727
Label shape    :  (56727, 2)
New rate       : 99.87%


In [124]:
for i in range(5):
    print(np.shape(training_examples[i]))

(95, 95, 3)
(95, 95, 3)
(95, 95, 3)
(95, 95, 3)
(95, 95, 3)


In [125]:
# cross validation
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(train, labels, test_size = 0.2)
print("TRAIN : ", train_images.shape, train_labels.shape)
print("TEST  : ", test_images.shape, test_labels.shape)

TRAIN :  (45351, 95, 95, 3) (45351,)
TEST  :  (11338, 95, 95, 3) (11338,)


In [126]:
train_images = train_images.reshape(train_images.shape[0], 
                                    train_images[0].shape[0], 
                                    train_images[0].shape[1], 
                                    train_images[0].shape[2])
print(train_images.shape)

(45351, 95, 95, 3)


In [131]:
# Convnet
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(95, 95, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (5, 5), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_97 (Conv2D)           (None, 91, 91, 32)        2432      
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 41, 41, 64)        51264     
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 16, 16, 128)       204928    
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 4, 4, 256)         819456    
__________

In [128]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
 2944/45351 [>.............................] - ETA: 14:13 - loss: 8.0308 - acc: 0.4963

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
def apply_padding(image_array, padding = 50):
    if len(image_array.shape) == 2:
        w, h = image_array.shape
        padded_array = np.zeros((w + padding * 2, h + padding * 2))
    else:
        w, h, c = image_array.shape
        padded_array = np.zeros((w + padding * 2, h + padding * 2, c))
        
    # Copy all content
    for i in range(w):
        for j in range (h):
            padded_array[i + padding][j + padding] = image_array[i][j]
            
    # For left and right
    for i in range(w):
        for j in range(padding):
            # Left
            padded_array[padding + i][padding - j] = image_array[i][j + 1]
            # Right
            padded_array[padding + i][h + padding + j] = image_array[i][h - j - 1]
            
    # For up and down
    for i in range(h):
        for j in range(padding):
            # Up
            padded_array[padding - j][padding + i] = image_array[j + 1][i]
            # Down
            padded_array[w + padding + j][padding + i] = image_array[w - j - 1][i]
            
    # For diagonal
    for i in range(padding):
        for j in range(padding):
            # Up Left
            padded_array[i][j] = image_array[padding - i][padding - j]
            # Up Right
            padded_array[i][h + padding + j] = image_array[padding - i][h - j - 1]
            # Downn Left
            padded_array[w + padding + i][j] = image_array[w - i - 1][padding - j]
            # Down Right
            padded_array[w + padding + i][h + padding + j] = image_array[w - i - 1][h - j - 1]
    
    return np.uint8(padded_array * 255)